# 自然数分割問題を計算する

自然数分割問題とは、ある自然数の集合を２つのグループA, Bに分割し、それぞれのグループ内の自然数の和が同じになるような分割方法を探す問題です。
これをwildqatを使用して解いてみます。


wildqatがインストールされていない場合は、環境に併せて以下のようにインストールしてください。



In [1]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 2.0MB/s 


必要なライブラリをimportし、オブジェクトをインスタンス化します。

In [0]:
import blueqat.opt as wq
import numpy as np
a = wq.opt()


解きたい問題のQUBOマトリクスを作成します。
N個の自然数の$i$番目の自然数を$n_i$とし、その自然数がどちらのグループに属するかを$q_i$で表します。
$n_i$がグループAに属する時には
$q_i=1$、グループBに属する時には$q_i=0$とします。
ここで、２つのグループ内のそれぞれの和が等しい時に最小となるようなコスト関数$E$を考えます。

この場合、

　$E=\{\sum_{i=1}^{N}n_i*(2q_i-1)\}^2$ 

とすれば、自然数$n_i$がグループＡに属するとき$2q_i-1=1$、グループBに属するとき$2q_i-1=-1$
になりますので、グループＡとグループＢに属する自然数の和が等しいときに
$E=0$になり、異なると$E>0$になります。

展開すると、

　$E=(\sum_{i=1}^{N}2n_iq_i)^2 -  2(\sum_{i=1}^{N}2n_iq_i)(\sum_{j=1}^{N}n_j) + (\sum_{i=1}^{N}n_i)^2$ 

コスト関数Eは最小化すれば良いので、最後の定数項は要らなくなります。またコスト関数は大きさのみ関係あるので、全体を４で割って

　$E=(\sum_{i=1}^{N}n_iq_i)^2 - \sum_{i=1}^{N}n_iq_i\sum_{j=1}^{N}n_j$ 

また、$q_i=1$または$q_i=0$のとき、$q_i^2=q_i$です。また、$\sum_{j=1}^N{n_j}$ はnの総和で定数ですので、
$n_{sum}$とします。さらに展開して整理すると</br>

　$E=\sum_{i=1}^{N}(n_i^2 - n_{sum}n_i)q_i +2 \sum_{i < j}n_in_jq_iq_j$ 

これを行列表記すると下記のようになります。

　$qubo = \left[\begin{array}{rrrrr}n_1^2 - n_{sum}n_1 & 2n_1n_2 & 2n_1n_3 & 2n_1n_4 & ...\\ 0 & n_2^2 - n_{sum}n_2 & 2n_2n_3& 2n_2n_4 &...\\ 0 & 0 & n_3^2 - n_{sum}n_3 & 2n_3n_4 & ...\\ 0 & 0 & 0 & n_4^2 - n_{sum}n_4 & ...\\ ... & ... & ... & ... &... \end{array} \right]$ 

これをpythonプログラムで書き、シミュレータを実行して結果を得ます。

In [0]:
numbers = np.array([3,2,6,9,2,5,7,3,3,6,7,3,5,3,2,2,2,6,8,4,6,3,3,6,4,3,3,2,2,5,8,9])
a.qubo = np.zeros((numbers.size,numbers.size))
for i in range(numbers.size):
  for j in range(numbers.size):
    if i == j:
      a.qubo[i][i]=numbers[i]**2-numbers.sum()*numbers[i]
    elif i<j:
      a.qubo[i][j]=2*numbers[i] * numbers[j]
answer = a.sa()

得られた結果を表示してみます。
自然数が２つのグループに分けられ、和が等しくなっています。

In [9]:
group1_string = ""
group2_string = ""
group1_sum = 0
group2_sum = 0
for i in range(numbers.size):
  if answer[i] == 0:
    group1_string+= '+' + str(numbers[i])
    group1_sum+=numbers[i]
  else:
    group2_string+= '+' + str(numbers[i])
    group2_sum+=numbers[i]

print(group1_string[1:],"=",group1_sum)
print(group2_string[1:],"=",group1_sum)

6+3+3+6+3+2+6+8+6+3+6+4+3+2+2+8 = 71
3+2+9+2+5+7+7+5+3+2+2+4+3+3+5+9 = 71
